In [1]:
import sys
import os
sys.path.append(os.path.abspath("../"))
print('\n'.join(sys.path))

/opt/conda/envs/slicechatter/lib/python310.zip
/opt/conda/envs/slicechatter/lib/python3.10
/opt/conda/envs/slicechatter/lib/python3.10/lib-dynload

/opt/conda/envs/slicechatter/lib/python3.10/site-packages
/root/chatter


In [2]:
from importlib import reload
import json
from tqdm.notebook import tqdm
from utils.config import *

In [3]:
# print(f"Counting {len(meta_info['idx_to_method_name'])} methods to test")
# create db connection
from pymongo import MongoClient

client = MongoClient(mongo_url, mongo_port)
# db = client[project_name]

In [22]:
from procedures import report
import utils.report

project_name = "batch-processing-gateway"
with open(os.path.join(playground_dir, project_name, "meta.json"), "r") as file:
    meta_info = json.load(file)
print(f"Counting {len(meta_info['idx_to_method_name'])} methods to test")
db = client[project_name]
cov_result = list([])

reload(report)
reload(utils.report)
for method_to_test in tqdm(meta_info['method_name_to_idx']):
    log_dir = os.path.join(playground_dir, project_name, 'methods_no_slice', meta_info['method_name_to_idx'][method_to_test])
    collection = db.get_collection(method_to_test)
    report.single_method_report(log_dir, collection, meta_info['put_path'], JACOCO_CLI, src_dir='src/main')
    cov_result.append(report.single_method_analyse(log_dir, db.get_collection(method_to_test)))

Counting 7 methods to test


  0%|          | 0/7 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "/root/chatter/procedures/report.py", line 96, in single_method_analyse
    coverage_result = jacoco_analysis(os.path.join(log_dir, "full_report"),
  File "/root/chatter/utils/report.py", line 57, in jacoco_analysis
    with open(html_path, "r") as file:
FileNotFoundError: [Errno 2] No such file or directory: '/root/experiments/playground/batch-processing-gateway/methods_no_slice/method_2/full_report/com.apple.spark.api/SubmissionSummary.html'
Traceback (most recent call last):
  File "/root/chatter/procedures/report.py", line 96, in single_method_analyse
    coverage_result = jacoco_analysis(os.path.join(log_dir, "full_report"),
  File "/root/chatter/utils/report.py", line 57, in jacoco_analysis
    with open(html_path, "r") as file:
FileNotFoundError: [Errno 2] No such file or directory: '/root/experiments/playground/batch-processing-gateway/methods_no_slice/method_6/full_report/com.apple.spark.tools/LoadTest.html'


In [23]:
def _report_slice(_cov_result, _n):
        _inst = [float(item[list(item.keys())[0]]['inst_cov'][:-1]) for item in _cov_result]
        _bran = [float(item[list(item.keys())[0]]['bran_cov'][:-1]) for item in _cov_result]
        return sum(_inst) / _n, sum(_bran) / _n
_report_slice(cov_result, 7)

(55.142857142857146, 50.142857142857146)

In [6]:
# slice cnt
import glob
projects = os.listdir(playground_dir)
for project in projects:
    n = len(glob.glob("**/steps/log_*.txt", root_dir=os.path.join(playground_dir, project, 'methods'), recursive=True))
    avg = n / len(os.listdir(os.path.join(playground_dir, project, 'methods')))
    print(f"{project}, {n}, {avg}")

commons-cli, 64, 10.666666666666666
commons-codec, 309, 16.263157894736842
batch-processing-gateway, 88, 12.571428571428571
jdom2, 278, 13.238095238095237
commons-collections, 199, 14.214285714285714
ruler, 285, 17.8125
commons-csv, 73, 12.166666666666666
gson, 288, 13.714285714285714
windward, 23, 11.5
datafaker, 81, 10.125


In [5]:
# batch report
import logging
puts = os.listdir(playground_dir)
print(f"playground: {playground_dir}")
for put in puts:
    try:
        with open(os.path.join(playground_dir, put, 'result.json'), 'r') as file:
            results = json.load(file)
        with open(os.path.join(playground_dir, put, "meta.json"), 'r') as file:
            meta = json.load(file)
            n = len(meta['method_name_to_idx'])
    except FileNotFoundError as e:
        logging.error(e)
        continue
        
    print(f"For put {put}: n={n}")
    def _report_slice(_cov_result, _n):
        _inst = [float(item[list(item.keys())[0]]['inst_cov'][:-1]) for item in _cov_result]
        _bran = [float(item[list(item.keys())[0]]['bran_cov'][:-1]) for item in _cov_result]
        return sum(_inst) / _n, sum(_bran) / _n
    print(_report_slice(results, n))# cnt pass rate

playground: /root/experiments/sysprompt
For put commons-cli: n=6
(47.0, 46.333333333333336)
For put commons-codec: n=19
(43.0, 38.63157894736842)
For put batch-processing-gateway: n=7
(43.714285714285715, 42.0)
For put jdom2: n=21
(26.238095238095237, 25.952380952380953)
For put commons-collections: n=14
(24.928571428571427, 21.5)
For put ruler: n=16
(3.9375, 1.75)
For put commons-csv: n=6
(30.5, 29.0)
For put gson: n=21
(25.047619047619047, 23.666666666666668)
For put windward: n=2
(1.5, 1.5)
For put datafaker: n=8
(17.25, 20.625)


In [ ]:
# compile & run counting
projects = os.listdir(playground_dir)
for project in projects:
    method_experiment_roots = glob.glob(os.path.join(playground_dir, project, 'methods', 'method_*', 'fixing'))
    print('\n'.join(method_experiment_roots))